# LSTM lanugage model - Fall 2016

The data used here is a treebank dataset for sentiment analysis ( based on the sentence structure ) available in [Stanford's website](https://nlp.stanford.edu/sentiment/). The texts are basically movie reviews from [Rotten Tomatoes](http://rottentomatoes.com) originally collected and published by Pang and Lee (2005), but this dataset has labels for every syntactically plausible phrase in thousands of sentences. Half of sentences were considered positive and the other half negative and labels were extracted from longer movie review. 

The [Stanford Parser](https://nlp.stanford.edu/software/lex-parser.shtml) was used as well.

** Import the data from txt files **

In [95]:

## Prof Bowman Class exercise - Random Seed used!!! (see below)

def load_sst_data(path):
    
    import re
    import random
    
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)  # Strip out the parse information 
            example['text'] = text[1:]
            data.append(example)
            
    print path
    print "\n ----> Line example in the tree-bank format\n", line
    sentiment_labels = [x.replace("(",'').replace(")",'') for x in re.findall('\(\d\s[\w]*\)',line)]
    print " ----> Line example sentiment format\n", sentiment_labels,"\n"
    print " ----> Line example full text format\n", text,"\n"
            
    random.seed(1)
    random.shuffle(data)
    return data


In [96]:

# Positive (3,4) and negative (0,1) to understand tree bank
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}
easy_label_map = {0:'very negative', 1:'negative', 2:'None', 3:'positive', 4:'very positive'}
print "\n", easy_label_map,"\n"
    
data_home = 'data/trees'

training_set = load_sst_data(data_home  + '/train.txt')
dev_set = load_sst_data(data_home  + '/dev.txt')
test_set = load_sst_data(data_home  + '/test.txt')




{0: 'very negative', 1: 'negative', 2: 'None', 3: 'positive', 4: 'very positive'} 

data/trees/train.txt

 ----> Line example in the tree-bank format
(1 (2 (2 In) (2 this)) (1 (2 case) (1 (1 zero) (2 .))))

 ----> Line example sentiment format
['2 In', '2 this', '2 case', '1 zero'] 

 ----> Line example full text format
 In this case zero . 

data/trees/dev.txt

 ----> Line example in the tree-bank format
(1 (1 (1 (0 (1 (2 Schaeffer) (0 (2 has) (0 (2 to) (0 (2 (2 find) (2 (2 some) (2 hook))) (0 (2 on) (1 (2 which) (1 (2 to) (0 (2 hang) (0 (2 his) (1 (1 (3 persistently) (1 useless)) (2 movies))))))))))) (2 ,)) (2 and)) (1 (2 it) (2 (2 (2 might) (2 (2 as) (3 well))) (2 (2 be) (2 (2 (2 the) (2 resuscitation)) (2 (2 of) (2 (2 the) (2 (2 middle-aged) (2 character))))))))) (2 .))

 ----> Line example sentiment format
['2 Schaeffer', '2 has', '2 to', '2 find', '2 some', '2 hook', '2 on', '2 which', '2 to', '2 hang', '2 his', '3 persistently', '1 useless', '2 movies', '2 and', '2 it', '2 migh

In [97]:

print "Example of one review\n\n", training_set[0]


Example of one review

{'text': 'Beneath the uncanny , inevitable and seemingly shrewd facade of movie-biz farce ... lies a plot cobbled together from largely flat and uncreative moments .'}


** Convert the data to index vectors**

To simplify implementation I am using length of 15 words per sentence. I'll use the mark of the start of each sentence with < S>, end with </ S>, mark tokens after </S> with a special word symbol < PAD>, and out-of-vocabulary words with < UNK>, for unknown. 

This vocabulary is constructed based on common words (more than 35 times repeated)


In [6]:
import collections
import numpy as np

def sentence_to_padded_index_sequence(datasets):
    '''Annotates datasets with feature vectors.'''
    
    START = "<S>"
    END = "</S>"
    END_PADDING = "<PAD>"
    UNKNOWN = "<UNK>"
    SEQ_LEN = 16
    
    # Extract vocabulary
    def tokenize(string):
        return string.lower().split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    vocabulary = set([word for word in word_counter if word_counter[word] > 35])
    vocabulary = list(vocabulary)
    vocabulary = [START, END, END_PADDING, UNKNOWN] + vocabulary
    print "vocabulary size", len(vocabulary)
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))
    indices_to_words = {v: k for k, v in word_indices.items()}
        
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['index_sequence'] = np.zeros((SEQ_LEN), dtype=np.int32)
            
            token_sequence = [START] + tokenize(example['text']) + [END]
            
            for i in range(SEQ_LEN):
                if i < len(token_sequence):
                    if token_sequence[i] in word_indices:  ## Words outside vocab are those with counts less than 25
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                else:
                    index = word_indices[END_PADDING]  ## If there are no more words then <PAD> is the feature written
                example['index_sequence'][i] = index
    return indices_to_words, word_indices
    
    
indices_to_words, word_indices = sentence_to_padded_index_sequence([training_set, dev_set, test_set])


vocabulary size 445


In [7]:

print training_set[0]
print "Words in the vocabulary =", len(word_indices)
#print [(k,word_indices[k]) for k in sorted(word_indices,key=word_indices.__getitem__)]


{'text': 'Beneath the uncanny , inevitable and seemingly shrewd facade of movie-biz farce ... lies a plot cobbled together from largely flat and uncreative moments .', 'index_sequence': array([  0,   3, 334,   3, 126,   3, 291,   3,   3,   3, 383,   3,   3,
       350,   3, 109], dtype=int32)}
Words in the vocabulary = 445


**LSTM -Language Model **

This model will have dropout on the non-recurrent connections. IT Computes sparse softmax cross entropy between logits and labels with "tf.nn.sparse_softmax_cross_entropy_with_logits" (measures the probability error in discrete classification tasks in which the classes are mutually exclusive - each entry is in exactly one class). 

For this case the "label" is the next word of a sentence, since **a language model** considers  probability distribution over sequences (sentences) by assigning probability $P(w_{1},... ,w_{m})$ to the whole sequence (relative likelihood).  Language modeling is used for speech recognition, machine translation, part-of-speech tagging, and others. We basically consider that the previous words (context) should be able to predict the new one.

$$P(w_{t}|{\text{context}})\,\forall t\in V$$


In [8]:
import tensorflow as tf

In [9]:

class LanguageModel:
    
    def __init__(self, vocab_size, sequence_length):
        
        # Define the hyperparameters
        self.learning_rate = 0.3  # Should be about right
        self.training_epochs = 100   # How long to train for - chosen to fit within class time
        self.display_epoch_freq = 1  # How often to test and print out statistics
        self.dim = 32  # The dimension of the hidden state of the RNN
        self.embedding_dim = 16  # The dimension of the learned word embeddings
        self.batch_size = 256    # Somewhat arbitrary - can be tuned, but often tune for speed, not accuracy
        self.vocab_size = vocab_size  # Defined by the file reader above
        self.sequence_length = sequence_length  # Defined by the file reader above
        self.keep_rate = 0.75  # Used in dropout (at training time only, not at sampling time)
    
        # Define the trained parameters.
        self.E = tf.Variable(tf.random_normal([self.vocab_size, self.embedding_dim], stddev=0.1))
        
        self.W = {'f': tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1)),
                  'i': tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1)),
                  'C': tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1)),
                  'o': tf.Variable(tf.random_normal([self.embedding_dim + self.dim, self.dim], stddev=0.1)),
                  'cl': tf.Variable(tf.random_normal([self.dim, vocab_size], stddev=0.1))
                  }
        
        self.b = {'f': tf.Variable(tf.random_normal([self.dim], stddev=0.1)),
                  'i': tf.Variable(tf.random_normal([self.dim], stddev=0.1)),
                  'C': tf.Variable(tf.random_normal([self.dim], stddev=0.1)),
                  'o': tf.Variable(tf.random_normal([self.dim], stddev=0.1)),
                  'cl': tf.Variable(tf.random_normal([vocab_size], stddev=0.1))
                  }

        # Define the input placeholder(s).
        self.x = tf.placeholder(tf.int32, [None,self.sequence_length])
        self.x_slices = tf.split( self.x,self.sequence_length, axis=1 )   # change 2017
        self.keep_rate_ph = tf.placeholder(tf.float32, [])
        
        # self.logits should contain one [batch_size, vocab_size]-shaped TF tensor of logits 
        #   for each step of the model.
        self.logits = []
        # self.costs should contain one [batch_size]-shaped TF tensor of cross-entropy loss 
        #   values for each step of the model.
        self.costs = []
        # self.h and c should each start contain one [batch_size, dim]-shaped TF tensor of LSTM
        #   activations for each of the lengthnumber+1 *states* of the model -- one tensor of zeros for the 
        #   starting state followed by one tensor each for the remaining lengthnumber steps.
        self.h_zero = tf.zeros( [self.batch_size, self.dim] )   # change 2017
        self.c_zero = tf.zeros( [self.batch_size, self.dim] )   # change 2017
        self.h = []
        self.c =[]
                
        # Define one step of the RNN for each word-place (index)
        def step(x, h_prev, c_prev):
            emb_x = tf.nn.embedding_lookup(self.E, x)
            emb_x = tf.nn.dropout(emb_x, self.keep_rate_ph)
            emb_h_prev = tf.concat(values=[emb_x, h_prev],axis=1)   # change 2017 instead of tf.concat(1,[emb_x, h_prev]) 
            f_t = tf.nn.sigmoid( tf.matmul(emb_h_prev, self.W['f'])  + self.b['f'] )
            i_t = tf.nn.sigmoid( tf.matmul(emb_h_prev, self.W['i'])  + self.b['i'] )
            C_tilde_t = tf.nn.tanh( tf.matmul(emb_h_prev, self.W['C'])  + self.b['C'])
            C_t = tf.multiply( f_t, c_prev) +  tf.multiply( i_t, C_tilde_t)     # change 2017  instead of "mul"
            o_t = tf.nn.sigmoid( tf.matmul(emb_h_prev, self.W['o'])  + self.b['o'] )
            h = o_t * tf.nn.tanh( C_t )
            return h, C_t
        
        h_prev = self.h_zero
        c_prev = self.c_zero
        
        # Start unrolling the text (sentence), each word will get the same process (step)
        for t in range(self.sequence_length-1):
            x_t = tf.reshape(self.x_slices[t], [-1])
            h_prev, c_prev = step(x_t, h_prev, c_prev)
            self.logit_t = tf.matmul(tf.nn.dropout(h_prev, self.keep_rate_ph), self.W['cl']) + self.b['cl']
            self.x_t_new = tf.reshape(self.x_slices[t+1], [-1])
            self.logits.append( self.logit_t )
            self.costs.append(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logit_t, labels=self.x_t_new ))
            self.h.append(h_prev)
            self.c.append(c_prev)
        
        
        #Sum costs for each word in each example, but average cost across examples.
        self.costs_tensor = tf.concat([tf.expand_dims(cost, 1) for cost in self.costs] , 1)
        self.cost_per_example = tf.reduce_sum(self.costs_tensor, 1)
        self.total_cost = tf.reduce_mean(self.cost_per_example)
            
        # This library call performs the main SGD update equation
        self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.total_cost)
        
        # Create an operation to fill zero values in for W and b
        self.init = tf.initialize_all_variables()
        
        # Create a placeholder for the session that will be shared between training and evaluation
        self.sess = None
        
                
    def train(self, training_data):
        
        def get_minibatch(dataset, start_index, end_index):
            indices = range(start_index, end_index)
            vectors = np.vstack([dataset[i]['index_sequence'] for i in indices])
            return vectors
        
        self.sess = tf.Session()
        
        self.sess.run(self.init)
        print 'Training.'

        # Training cycle
        for epoch in range(self.training_epochs):
            random.shuffle(training_set)
            avg_cost = 0.
            total_batch = int(len(training_set) / self.batch_size)
            
            # Loop over all batches in epoch
            for i in range(total_batch):
                # Assemble a minibatch of the next B examples: we ask for the indeex S
                # EQUENCE OF EACH OBSERVATION, BASICALLY VECTORS WITH THE NUMBERS (INDEX TO WORDS)
                minibatch_vectors = get_minibatch(training_set, self.batch_size * i, self.batch_size * (i + 1))

                # Run the optimizer to take a gradient step, and also fetch the value of the 
                # cost function for logging
                
                # Example of how I PRINT actual values
                #xslices,x_new, logit_t = self.sess.run([self.x_slices, self.x_t_new, self.logit_t ],
                #                                       feed_dict={self.x: minibatch_vectors})
                #print "x slices", xslices[0], "x new", x_new, "logit", logit_t

                _, c = self.sess.run( [self.optimizer, self.total_cost], 
                                     feed_dict= { self.x: minibatch_vectors, self.keep_rate_ph:self.keep_rate } )

                # Compute average loss
                avg_cost += c / (total_batch * self.batch_size)
                
            # Display some statistics about the step
            if (epoch+1) % self.display_epoch_freq == 0:
                print "Epoch:", (epoch+1), "Cost:", avg_cost, "Sample:", self.sample()
    
    
    def sample(self):
        # This samples a sequence of tokens from the model starting with <S>.
        # We only ever run the first timestep of the model, and use an effective batch size of one
        # but we leave the model unrolled for multiple steps, and use the full batch size to simplify 
        # the training code. This slows things down.

        def brittle_sampler():
            # The main sampling code. Can fail randomly due to rounding errors that yield probibilities
            # that don't sum to one.
            
            word_indices = [0]    # 0 here is the "<S>" symbol
            for i in range(self.sequence_length - 1):
                dummy_x = np.zeros((self.batch_size, self.sequence_length))
                dummy_x[0][0] = word_indices[-1]
                feed_dict = {self.x: dummy_x, self.keep_rate_ph:1.0}
                if i > 0:
                    feed_dict[self.h_zero] = h
                    feed_dict[self.c_zero] = c           
                h, c, logits = self.sess.run([self.h[1], self.c[1], self.logits[0]], 
                                             feed_dict=feed_dict)  
                logits = logits[0, :]    # Discard all but first batch entry
                exp_logits = np.exp(logits - np.max(logits))
                distribution = exp_logits / exp_logits.sum()
                sampled_index = np.flatnonzero(np.random.multinomial(1, distribution))[0]
                word_indices.append(sampled_index)
            words = [indices_to_words[index] for index in word_indices]
            return ' '.join(words)
        
        while True:
            try:
                sample = brittle_sampler()
                return sample
            except ValueError as e:  # Retry if we experience a random failure.
                pass
            

In [10]:

##Agan, this is for 15 words per sentence

model = LanguageModel(len(word_indices), 16)
model.train(training_set)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Training.
Epoch: 1 Cost: 0.241791106535 Sample: <S> audience <UNK> might <UNK> <UNK> <UNK> <UNK> </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 2 Cost: 0.20780012147 Sample: <S> while <UNK> , <UNK> , works when feeling american to the <UNK> feels is come
Epoch: 3 Cost: 0.200529943361 Sample: <S> wit , set the but <UNK> <UNK> funny this <UNK> which <UNK> <UNK> . </S>
Epoch: 4 Cost: 0.196078305895 Sample: <S> all we <UNK> you good fans <UNK> yet a i <UNK> life in <UNK> <UNK>
Epoch: 5 Cost: 0.193033874938 Sample: <S> the <UNK> , in a <UNK> make 's probably . <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 6 Cost: 0.190171249888 Sample: <S> worst <UNK> does in be melodrama <UNK> short much the <UNK> as obvious . </S>
Epoch: 7 Cost: 0.188497461604 Sample: <S> the like this <UNK> <UNK> with need <UNK> just the film of this a <UNK>
Epoch: 8 Cost: 0.186405470877 Sample: <S> worst ... a first <UNK> <UNK> <UNK> <UNK> of this t

Epoch: 70 Cost: 0.166334387931 Sample: <S> charming in the such half moments matter under the film itself is <UNK> <UNK> human
Epoch: 71 Cost: 0.166583486579 Sample: <S> a dark and smart , a <UNK> its film i <UNK> 're <UNK> is the
Epoch: 72 Cost: 0.166238445676 Sample: <S> it 's ... is for i <UNK> is too original , then has <UNK> <UNK>
Epoch: 73 Cost: 0.166063136675 Sample: <S> this <UNK> is n't enough than these film 's <UNK> 's <UNK> from the <UNK>
Epoch: 74 Cost: 0.1661427012 Sample: <S> a <UNK> into point movies <UNK> has something <UNK> in a <UNK> <UNK> , <UNK>
Epoch: 75 Cost: 0.166248863394 Sample: <S> too <UNK> <UNK> may <UNK> and <UNK> only a <UNK> are <UNK> <UNK> do n't
Epoch: 76 Cost: 0.165936338179 Sample: <S> with <UNK> to take some of sometimes <UNK> -rrb- else is like those <UNK> <UNK>
Epoch: 77 Cost: 0.165914332776 Sample: <S> this is i dialogue , <UNK> <UNK> and nothing also just is boring , -lrb-
Epoch: 78 Cost: 0.165901827993 Sample: <S> turns to <UNK> and <UNK> any <

In [11]:
print model.sample()
print model.sample()
print model.sample()
print model.sample()
print model.sample()
print model.sample()

<S> the performances has its <UNK> action is so <UNK> <UNK> , <UNK> a only <UNK>
<S> <UNK> as <UNK> do <UNK> a <UNK> movie about the <UNK> high <UNK> makes also
<S> if seems about <UNK> <UNK> , the film is nothing of it 's do n't
<S> <UNK> with everything that makes its <UNK> could have it 's <UNK> so to modern
<S> it 's most of a <UNK> <UNK> with the <UNK> mr. most ` <UNK> ,
<S> it about <UNK> <UNK> proves ... there 's <UNK> to <UNK> <UNK> <UNK> <UNK> <UNK>


## Additional notes about RNN and Deep Learning

https://gigaom.com/2013/08/16/were-on-the-cusp-of-deep-learning-for-the-masses-you-can-thank-google-later/
"**Word2vec** is prepackaged deep-learning software released by google and designed to understand the **relationships between words** with no human guidance. Just input a textual data set and let underlying predictive models get to work learning. [Word2vec](https://opensource.googleblog.com/2013/08/learning-meaning-behind-words.html) uses distributed representations of text to capture similarities among concepts. For example, it understands that Paris and France are related the same way Berlin and Germany are (capital and country), and not the same way Madrid and Italy are. This chart shows how well it can learn the concept of capital cities, just by reading lots of news articles -- with no human supervision"


http://neuralnetworksanddeeplearning.com/chap5.html 
" In general, neurons in the earlier layers learn much more slowly than neurons in later layers. And while we've seen this in just a single network, there are fundamental reasons why this happens in many neural networks. The phenomenon is known as the **vanishing gradient** problem. Why does the vanishing gradient problem occur? Are there ways we can avoid it? And how should we deal with it in training deep neural networks? In fact, we'll learn shortly that it's not inevitable, although the alternative is not very attractive, either: sometimes the gradient gets much larger in earlier layers! This is the exploding gradient problem, and it's not much better news than the vanishing gradient problem. More generally, it turns out that the gradient in deep neural networks is unstable, tending to either explode or vanish in earlier layers. This instability is a fundamental problem for gradient-based learning in deep neural networks. It's something we need to understand, and, if possible, take steps to address. "

http://link.springer.com/chapter/10.1007%2F11840817_26#page-1
" Introduced in Bengio et al. (1994), the **exploding gradients** problem refers to the large increase in the norm
of the gradient during training. Such events are due to the explosion of the long term components, which can grow exponentially more than short term ones. The vanishing gradients problem refers to the opposite behaviour, when long term components go exponentially fast to norm 0, making it impossible for the model to learn correlation between temporally distant events. 

Using an **L1 or L2 penalty** on the recurrent weights can help with exploding gradients. Assuming weights are initialized to small values, the largest singular value λ1 of Wrec is probably smaller than 1. The L1/L2 term can ensure that during training λ1 stays smaller than 1, and in this regime gradients can not explode. This approach limits the model to single point attractor at the origin, where any information inserted in the model dies out exponentially fast. This prevents the model to learn generator networks, nor can it exhibit long term memory traces. "
